## NiN 块的结构

NiN 块由 3 个卷积层构成

其中三个卷积层都没有池化层

第一个卷积层可以改变输入数据的通道数

后面两个卷积层不能改变输入数据的通道数，且卷积核的大小为 1 * 1

后面两个卷积层相当于对输入数据在每一个像素上进行不同通道的全连接（输入层神经元数与输出层神经元数相同）

<br>

## NiN 块的示例

In [3]:
from torch import nn

def NiN_BLOCK(in_channels, out_channels, kernel_size, strides, padding):
    layers = []

    # 卷积计算层
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding))
    layers.append(nn.ReLU())

    # 针对不同通道的同一个像素点的全连接
    layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=1))
    layers.append(nn.ReLU())

    # 针对不同通道的同一个像素点的全连接（对于被 ReLU 函数激活后的值再进行一次全连接，提取更多复杂特征）
    layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=1))
    layers.append(nn.ReLU())
    
    return layers

<br>

## NiN 的结构

NiN 由多个 NiN 块组成

每一个 NiN 块之间衔接一个最大池化层

最后一个 NiN 块后面接一个全局平均池化层

NiN 没有全连接层，输出数据量等于最后一个池化层输出数据的通道数

<br>

## NiN 的一个示例

In [4]:
class NiN(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.Sequential(
            # 多个 NiN 块
            *NiN_BLOCK(1, 96, 11, 4, 0),
            nn.MaxPool2d(3, stride=2),
            *NiN_BLOCK(96, 256, 5, 1, 2),
            nn.MaxPool2d(3, stride=2),
            *NiN_BLOCK(256, 384, 3, 1, 1),
            nn.MaxPool2d(3, stride=2),

            # 进行一次 Dropout，防止过拟合
            nn.Dropout(0.5),

            # 最后一个 NiN 块将通道数变为所需要的输出数据量
            *NiN_BLOCK(384, 10, 3, 1, 1),

            # 通过自适应平均池化和拉平层实现全局平均池化的效果
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten()
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [5]:
import torch
from torch import nn


x = torch.randn(1, 2, 2, 2)
x

tensor([[[[ 0.6721, -1.0807],
          [ 0.6824, -1.1672]],

         [[-2.0617,  1.7322],
          [ 1.3500, -0.4090]]]])

In [6]:
dropout = nn.Dropout(0.5)
dropout(x)

tensor([[[[ 1.3442, -0.0000],
          [ 1.3648, -2.3344]],

         [[-4.1234,  0.0000],
          [ 0.0000, -0.0000]]]])

In [7]:
dropout2 = nn.Dropout2d(0.5)
dropout2(x)

tensor([[[[ 0.0000, -0.0000],
          [ 0.0000, -0.0000]],

         [[-4.1234,  3.4644],
          [ 2.7000, -0.8180]]]])